# Meeting Bot
Created by: Steven David Ong

#### Objective:

The client aims to optimize meetings by improving efficiency. To achieve this, participants are required to review the meeting brief and agenda items prior to the meeting. Upon review, participants must respond to the email thread indicating their agreement or disagreement with each agenda item.

A bot has been developed to streamline this process, consisting of four functions:

1. Send_email_to_recipient(): This function drafts and sends the email with the necessary attachments to the recipients.
2. Incoming_emails_handler(): When invoked, the bot will scan through the email inbox and extract data based on specific subject headers and item labels.
3. Voting_results(votes): This function summarizes the data extracted by the Incoming_emails_handler() function, tabulating the number of agrees and disagrees for each agenda item. Additionally, it parses out the names of the participants who do not agree.
5. Analyze_votes(): This function invokes both the Incoming_emails_handler() and Voting_results(votes) functions.

#### Design Brief:

Two different programs have been written to customize towards client's style of meeting. <br>

(a) On the spot poll - It allows participants to conduct a poll during the meeting, and the results are tabulated in real-time. This program is designed to streamline the decision-making process during the meeting and make it more efficient. <br>

(b) Poll with Delayed response - This program enables the user to send out an email to the participants using the first set of modularized code - *<font color='green'>Send_email_to_recipient()</font>*. The participants can then review the meeting brief and agenda items at their convenience and respond with their agreement or disagreement. The number of agreements and disagreements are tracked using the modularized codes  *<font color='green'>Incoming_emails_handler()</font>* and *<font color='green'>Voting_results(votes)</font>*. This program is designed to be more flexible, allowing participants to respond at their convenience, without interrupting their work schedule.

Both programs are designed to help the client achieve their objective of optimizing meetings and improving efficiency. Depending on the meeting format, the client can choose which program to use to suit their needs.

In [1]:
### ensure the following imports are done:
import smtplib
import email
import imaplib
import requests
import collections
from bs4 import BeautifulSoup
import re
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email import encoders
import pandas as pd

### Note to user
*Fill up the details in the [ ]*

#### (a) On the spot poll

In [2]:
smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_username = "[User_email]"
smtp_password = "[Password]"
recipient_email = "[recipient's email. placeholder, code will be invoked with emails in _name_]"


def send_email_to_recipients(to, subject, body, attachment_path=None):
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = recipient_email
    msg['Subject'] = subject
    
    
    msg.attach(MIMEText(body, 'html'))
    
    if attachment_path:
        with open(attachment_path, "rb") as attachment:
            # Create a MIMEBase object to hold the attachment
            attach = MIMEBase("application", "octet-stream")
            attach.set_payload(attachment.read())
       
    # Encode the attachment and add it to the message
        encoders.encode_base64(attach)
        attach.add_header("Content-Disposition", f"attachment; filename={attachment_path}")
        msg.attach(attach)    
    
    #### put in the attachment here
    attachment_path = ""
    
    server = smtplib.SMTP("smtp.gmail.com", smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)
    server.sendmail(recipient_email, to, msg.as_string())
    server.quit()

def incoming_emails_handler():
    mail = imaplib.IMAP4_SSL(imap_url)
    mail.login(smtp_username, smtp_password)
    mail.select('inbox')

    _, data = mail.search(None, 'OR (UNSEEN) (SEEN)', f'SUBJECT "Re: Meeting Minutes Voting"')
    email_ids = data[0].split()
    votes = []

    for email_id in email_ids:
        _, email_data = mail.fetch(email_id, '(RFC822)')
        raw_email = email_data[0][1]
        parsed_email = email.message_from_bytes(raw_email)
        
        from_email = email.utils.parseaddr(parsed_email['From'])[1]
        subject = str(parsed_email['Subject'])

        if subject and subject.startswith("Re: Meeting Minutes Voting"):
            body = ""
            if parsed_email.is_multipart():
                for part in parsed_email.walk():
                    if part.get_content_type() == "text/plain":
                        body = part.get_payload(decode=True).decode('utf-8')
                        break
            else:
                body = parsed_email.get_payload(decode=True).decode('utf-8')

            votes_raw = body.split('\n')
            votes_filtered = [vote.strip() for vote in votes_raw if vote.strip()]

            for vote in votes_filtered:
                match = re.match(r'Item (\d+): (Yes|No)', vote, re.IGNORECASE)
                if match:
                    item, decision = match.groups()
                    votes.append((from_email, int(item), decision.lower()))
                else:
                    continue
                    ## following is to fault test the loop
                    #print(f"Skipping non-vote email: '{vote}'")
    
    print(f"Retrieved {len(votes)} votes.")
    
    return votes

# Gmail server settings
imap_url = 'imap.gmail.com'
smtp_url = 'smtp.gmail.com'
smtp_port = 587

def voting_results(votes):
    counter = collections.Counter(votes)
    results = {}
    for from_email, item, decision in counter:
        if item not in results:
            results[item] = {'yes': 0, 'no': 0, 'no_voters': []}
        results[item][decision] += counter[(from_email, item, decision)]
        if decision == 'no':
            results[item]['no_voters'].append(from_email)

    data_captured = []            
    for item, decisions in results.items():
        no_voters = ', '.join(decisions['no_voters'])
        print(f"Agenda Item {item}: Yes - {decisions['yes']}, No - {decisions['no']} (Disagreed: {no_voters})")       
        no_voters = ', '.join(decisions['no_voters'])
        data_captured.append([item, decisions['yes'], decisions['no'], no_voters])
        
    df = pd.DataFrame(data_captured, columns=['agenda_item', 'number_of_yes', 'number_of_no', 'who_disagreed'])
    print(df)
    return df
        
        
if __name__ == "__main__":
    # Send the meeting minutes and voting instructions
    ### change the email brief and tailor accordingly
    recipients = ['recipient email 1', 'recipient email 2..., 3, 4, 5']
    subject = "Meeting Minutes Voting"
    body = """\
    <html>
    <body>
    <p>Please reply to this email with your vote on each agenda item. Use the format "Item X: Yes" or "Item X: No".</p>
    <p><b>Agenda Items:</b></p>
    <p>Item 1: Budget approval</p>
    <p>Item 2: Office relocation</p>
    <p>Item 3: New hires</p>
    </body>
    </html>
    """
    
    attachment_path = "W-8BEN Form.pdf"
    
    
    for recipient in recipients:
        send_email_to_recipients(recipient, subject, body, attachment_path)

    input("Press Enter after waiting for responses from recipients for full code to run...")

    # Retrieve and handle incoming email votes
    votes = incoming_emails_handler()
    print(f"Retrieved {len(votes)} votes.")

    # Display the voting results
    if votes:
        voting_results(votes)
    else:
        print("No new votes received.")

Press Enter after waiting for responses from recipients for full code to run... 


Retrieved 15 votes.
Retrieved 15 votes.
Agenda Item 1: Yes - 3, No - 2 (Disagreed: steven.onghb@gmail.com, steveong@umich.edu)
Agenda Item 2: Yes - 0, No - 5 (Disagreed: steven.onghb@gmail.com, ppppdde@outlook.sg, steveong@umich.edu)
Agenda Item 3: Yes - 2, No - 3 (Disagreed: ppppdde@outlook.sg, steveong@umich.edu)
   agenda_item  number_of_yes  number_of_no  \
0            1              3             2   
1            2              0             5   
2            3              2             3   

                                       who_disagreed  
0         steven.onghb@gmail.com, steveong@umich.edu  
1  steven.onghb@gmail.com, ppppdde@outlook.sg, st...  
2             ppppdde@outlook.sg, steveong@umich.edu  


#### (b) Poll with Delayed response 

In [3]:
smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_username = "[User_email]"
smtp_password = "[Password]"
recipient_email = "[recipient's email. placeholder, code will be invoked with emails in _name_]"


def send_email_to_recipients(to, subject, body, attachment_path=None):
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = recipient_email
    msg['Subject'] = subject
    
    
    msg.attach(MIMEText(body, 'html'))
    
    if attachment_path:
        with open(attachment_path, "rb") as attachment:
            # Create a MIMEBase object to hold the attachment
            attach = MIMEBase("application", "octet-stream")
            attach.set_payload(attachment.read())
       
    # Encode the attachment and add it to the message
        encoders.encode_base64(attach)
        attach.add_header("Content-Disposition", f"attachment; filename={attachment_path}")
        msg.attach(attach)    
    
    #### put in the attachment here
    attachment_path = ""
    
    server = smtplib.SMTP("smtp.gmail.com", smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)
    server.sendmail(recipient_email, to, msg.as_string())
    server.quit()
    
    
if __name__ == "__main__":
    # Send the meeting minutes and voting instructions
    recipients = ['steven.onghb@gmail.com', 'ppppdde@outlook.sg', 'alisonanna@gmail.com', 'steveong@umich.edu']
    subject = "Meeting Minutes Voting"
    body = """\
    <html>
    <body>
    <p>Please reply to this email with your vote on each agenda item. Use the format "Item X: Yes" or "Item X: No".</p>
    <p><b>Agenda Items:</b></p>
    <p>Item 1: Budget approval</p>
    <p>Item 2: Office relocation</p>
    <p>Item 3: New hires</p>
    </body>
    </html>
    """

    attachment_path = "eg. W-8BEN Form.pdf"

    for recipient in recipients:
        send_email_to_recipients(recipient, subject, body, attachment_path)    

In [4]:
# Replace with your Gmail credentials
smtp_username = "[User_email]"
smtp_password = "[Password]"

# Gmail server settings
imap_url = 'imap.gmail.com'
smtp_url = 'smtp.gmail.com'
smtp_port = 587


def incoming_emails_handler():
    mail = imaplib.IMAP4_SSL(imap_url)
    mail.login(smtp_username, smtp_password)
    mail.select('inbox')

    _, data = mail.search(None, 'OR (UNSEEN) (SEEN)', f'SUBJECT "Re: Meeting Minutes Voting"')
    
    ### alternatively
    ### _, data = mail.search(None, 'UNSEEN') if just to count new incoming votes
    
    email_ids = data[0].split()
    votes = []

    for email_id in email_ids:
        _, email_data = mail.fetch(email_id, '(RFC822)')
        raw_email = email_data[0][1]
        parsed_email = email.message_from_bytes(raw_email)
        
        from_email = email.utils.parseaddr(parsed_email['From'])[1]
        subject = str(parsed_email['Subject'])

        if subject and subject.startswith("Re: Meeting Minutes Voting"):
            body = ""
            if parsed_email.is_multipart():
                for part in parsed_email.walk():
                    if part.get_content_type() == "text/plain":
                        body = part.get_payload(decode=True).decode('utf-8')
                        break
            else:
                body = parsed_email.get_payload(decode=True).decode('utf-8')

            votes_raw = body.split('\n')
            votes_filtered = [vote.strip() for vote in votes_raw if vote.strip()]

            for vote in votes_filtered:
                match = re.match(r'Item (\d+): (Yes|No)', vote, re.IGNORECASE)
                if match:
                    item, decision = match.groups()
                    votes.append((from_email, int(item), decision.lower()))
                else:
                    continue
                    ## following is to fault test the loop if ever required to fault test
                    #print(f"Skipping non-vote email: '{vote}'")
    
    print(f"Retrieved {len(votes)} votes.")
    
    return votes

def voting_results(votes):
    counter = collections.Counter(votes)
    results = {}
    for from_email, item, decision in counter:
        if item not in results:
            results[item] = {'yes': 0, 'no': 0, 'no_voters': []}
        results[item][decision] += counter[(from_email, item, decision)]
        if decision == 'no':
            results[item]['no_voters'].append(from_email)

    data_captured = []            
    for item, decisions in results.items():
        no_voters = ', '.join(decisions['no_voters'])
        print(f"Agenda Item {item}: Yes - {decisions['yes']}, No - {decisions['no']} (Disagreed: {no_voters})")       
        no_voters = ', '.join(decisions['no_voters'])
        data_captured.append([item, decisions['yes'], decisions['no'], no_voters])
        
    response_df = pd.DataFrame(data_captured, columns=['agenda_item', 'number_of_yes', 'number_of_no', 'who_disagreed'])
    #print(response_df)
    return response_df


In [5]:
def analyze_votes():
    # Retrieve and handle incoming email votes
    votes = incoming_emails_handler()
    print(f"Retrieved {len(votes)} votes.")
    

    # Display the voting results
    if votes:
        return voting_results(votes)
    else:
        print("No new votes received.")


In [6]:
analyze_votes()

Retrieved 15 votes.
Retrieved 15 votes.
Agenda Item 1: Yes - 3, No - 2 (Disagreed: steven.onghb@gmail.com, steveong@umich.edu)
Agenda Item 2: Yes - 0, No - 5 (Disagreed: steven.onghb@gmail.com, ppppdde@outlook.sg, steveong@umich.edu)
Agenda Item 3: Yes - 2, No - 3 (Disagreed: ppppdde@outlook.sg, steveong@umich.edu)


,agenda_item,number_of_yes,number_of_no,who_disagreed
0,1,3,2,"steven.onghb@gmail.com, steveong@umich.edu"
1,2,0,5,"steven.onghb@gmail.com, ppppdde@outlook.sg, st..."
2,3,2,3,"ppppdde@outlook.sg, steveong@umich.edu"
